In [ ]:
Se definen las funciones a utilizar:

In [ ]:
# %% IMPORTACIONES Y DEFINICIONES
import scipy.signal as signal
import math 
import biosppy
from biosppy.signals import ecg as ekg
import numpy as np
import matplotlib.pyplot as plt
import pyedflib
import nolds
import neurokit2 as nk
from datetime import datetime, timedelta
import pandas as pd

#defino las funciones
def abrir_y_corroborar(nombre_archivo, canal, freq_muestreo):
    ruta_archivo_edf = nombre_archivo
    archivo_edf = pyedflib.EdfReader(ruta_archivo_edf)
    cantidad_canales = archivo_edf.signals_in_file 
    num_muestras = archivo_edf.getNSamples()[canal]  
    frecuencia_muestreo = 512
    datos_canal = archivo_edf.readSignal(canal)
    eje_temp=np.arange(0, num_muestras/frecuencia_muestreo, 1/frecuencia_muestreo)
    archivo_edf.close()
    
    out = ekg.ecg(signal=datos_canal, sampling_rate=frecuencia_muestreo, show=True)
    senial_filt = out['filtered']
    rpeaks = out['rpeaks']
    heart_rate_ts0 = out['heart_rate_ts'] #LOS TIEMPOS EN SEGUNDOS EN QUE SE CALCULO 
    heart_rate0 = out['heart_rate']       #EL VALOR DE FRECUENCIA CARDIACA
    eje_temp0 = out['ts'] 
    plt.plot(eje_temp[5000:5500], senial_filt[5000:5500])
    return rpeaks, eje_temp, senial_filt

def calcular_segundos(hora_inicio, hora_evento, mismo_dia=True):
    # Convertir las horas a objetos datetime
    inicio = datetime.strptime(hora_inicio, "%H.%M.%S")
    evento = datetime.strptime(hora_evento, "%H.%M.%S")
    
    # Si el evento ocurre al día siguiente, sumamos 1 día al evento
    if not mismo_dia:
        evento += timedelta(days=1)
    
    # Calcular la diferencia en segundos
    segundos_transcurridos = int((evento - inicio).total_seconds())
    
    return segundos_transcurridos

def ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal):
    clasificacion = []
    ventanas_rpeaks = []
    for i in range(0, len(rpeaks), ventana_len - ventana_solap):
        # extraigo la ventana de la señal
        ventana_rpeaks = rpeaks[i:i + ventana_len]    #la ultima ventana puede tener longitud menor
        # Extraer la señal correspondiente en senial_filtrada
        ventana_ecg = ecg[ventana_rpeaks[0]:ventana_rpeaks[-1] + 1]    
        if (len(ventana_rpeaks) == ventana_len):
            ventanas_rpeaks.append(ventana_rpeaks)
            if (i + ventana_len < len(rpeaks) and rpeaks[i] > muestra_inicial_preictal and rpeaks[i+ventana_len] < muestra_final_preictal):
                clasificacion.append('preictal')
            else:
                    clasificacion.append('no_preictal')
    return ventanas_rpeaks, clasificacion

def ventanear_2(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal1, muestra_final_preictal1, muestra_inicial_preictal2, muestra_final_preictal2):
    clasificacion = []
    ventanas_rpeaks = []
    for i in range(0, len(rpeaks), ventana_len - ventana_solap):
        # extraigo la ventana de la señal
        ventana_rpeaks = rpeaks[i:i + ventana_len]    #la ultima ventana puede tener longitud menor
        # Extraer la señal correspondiente en senial_filtrada
        ventana_ecg = ecg[ventana_rpeaks[0]:ventana_rpeaks[-1] + 1]    
        if (len(ventana_rpeaks) == ventana_len):
            ventanas_rpeaks.append(ventana_rpeaks)
            if (i + ventana_len < len(rpeaks) and ((rpeaks[i] > muestra_inicial_preictal1 and rpeaks[i+ventana_len] < muestra_final_preictal1) 
                                                or (rpeaks[i] > muestra_inicial_preictal2 and rpeaks[i+ventana_len] < muestra_final_preictal2))):
                clasificacion.append('preictal')
            else:
                    clasificacion.append('no_preictal')
    return ventanas_rpeaks, clasificacion
def ventanear_3(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal1, muestra_final_preictal1, muestra_inicial_preictal2, muestra_final_preictal2, muestra_inicial_preictal3, muestra_final_preictal3):
    clasificacion = []
    ventanas_rpeaks = []
    for i in range(0, len(rpeaks), ventana_len - ventana_solap):
        # extraigo la ventana de la señal
        ventana_rpeaks = rpeaks[i:i + ventana_len]    #la ultima ventana puede tener longitud menor
        # Extraer la señal correspondiente en senial_filtrada
        ventana_ecg = ecg[ventana_rpeaks[0]:ventana_rpeaks[-1] + 1]    
        if (len(ventana_rpeaks) == ventana_len):
            ventanas_rpeaks.append(ventana_rpeaks)
            if (i + ventana_len < len(rpeaks) and ((rpeaks[i] > muestra_inicial_preictal1 and rpeaks[i+ventana_len] < muestra_final_preictal1) 
                                                or (rpeaks[i] > muestra_inicial_preictal2 and rpeaks[i+ventana_len] < muestra_final_preictal2))
                                                or (rpeaks[i] > muestra_inicial_preictal3 and rpeaks[i+ventana_len] < muestra_final_preictal3)):
                clasificacion.append('preictal')
            else:
                    clasificacion.append('no_preictal')
    return ventanas_rpeaks, clasificacion
def calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion):
    # esta funcion es larga
    # Temporales
    time_features = []
    nni = []
    hr_mean = []
    hr_min = []
    hr_max = []
    hr_std = []
    p_nn_50 = []
    for i in range (len(ventanas_intervalos_nn)):
        #time_features.append(td.time_domain(ventanas_intervalos_nn[i]))
        #nni.append(time_features[i]['nni_diff_mean'])
        nni.append(td.nni_differences_parameters(nni=ventanas_intervalos_nn[i])['nni_diff_mean'])
        aux = td.hr_parameters(nni=ventanas_intervalos_nn[i])
        hr_mean.append(aux['hr_mean'])
        hr_min.append(aux['hr_mean'])
        hr_max.append(aux['hr_max'])
        hr_std.append(aux['hr_std'])
        aux = td.nn50(nni=ventanas_intervalos_nn[i])
        p_nn_50.append(aux['pnn50'])
    # Frecuenciales
    freq_features = []
    VLF_power = []
    lf_power = []
    hf_power = []
    for i in range (len(ventanas_intervalos_nn)):
        freq_features.append(fd.welch_psd(ventanas_intervalos_nn[i]))
        VLF_power.append(freq_features[i]['fft_abs'][0]) #0,1,2 porque asi lo hace la libreria esta
        lf_power.append(freq_features[i]['fft_abs'][1])
        hf_power.append(freq_features[i]['fft_abs'][2])
    # No lineales
    nonlinear_features = []
    sd1 = []
    sd2 = []
    sd_ratio = []
    elipse_area = []
    entropia = []

    for i in range (len(ventanas_intervalos_nn)):
        nonlinear_features.append(nl.poincare(ventanas_intervalos_nn[i]))
        sd1.append(nonlinear_features[i]['sd1'])
        sd2.append(nonlinear_features[i]['sd2'])
        sd_ratio.append(nonlinear_features[i]['sd_ratio'])
        elipse_area.append(nonlinear_features[i]['ellipse_area']) 
        aux = nl.sample_entropy(nni = ventanas_intervalos_nn[i])
        entropia.append(aux['sampen'])
        
    # Armo el DataFrame
    data = {
        'nni_diff_mean': [nni[i] for i in range(len(nni))],
        'hr_mean': [hr_mean[i] for i in range(len(hr_mean))],
        'hr_max': [hr_max[i] for i in range(len(hr_max))],
        'hr_std': [hr_std[i] for i in range(len(hr_std))],
        'pnn50' : [p_nn_50[i] for i in range(len(p_nn_50))],
        'VLF_power': [VLF_power[i] for i in range(len(VLF_power))],
        'LF_power': [lf_power[i] for i in range(len(lf_power))],
        'HF_power': [hf_power[i] for i in range(len(hf_power))],
        'sd1': [sd1[i] for i in range(len(sd1))],
        'sd2': [sd2[i] for i in range(len(sd2))],
        'sd_ratio': [sd_ratio[i] for i in range(len(sd_ratio))],
        'elipse_area': [elipse_area[i] for i in range(len(elipse_area))],
        'SampEn': [entropia[i] for i in range(len(entropia))],
        'clasificacion': [clasificacion[i] for i in range(len(clasificacion))]
    }
    df = pd.DataFrame(data)
    with pd.ExcelWriter(nombre_archivo_excell, mode='a', engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name=nombre_paciente, index=False)
def guardar_vectores_en_txt(vector1, vector2, vector3, nombre_archivo_txt):
    # Asegurarse de que los vectores solo contengan enteros o flotantes
    nombre_archivo_ecg = nombre_archivo_txt+'.ecg'
    nombre_archivo_txt=nombre_archivo_txt+'.txt'
    vector1 = [int(i) for i in vector1]
    vector2 = [float(i) for i in vector2]

    # Guardar los elementos de los primeros dos vectores en el archivo de texto
    with open(nombre_archivo_txt, 'w') as archivo:
        # Guardar el primer vector (enteros)
        for elemento in vector1:
            archivo.write(f"{elemento}\n")
        
        # Agregar una línea en blanco
        archivo.write("\n")
        
        # Guardar el segundo vector (flotantes)
        for elemento in vector2:
            archivo.write(f"{elemento}\n")
    
    # Guardar el tercer vector (ECG) en un archivo binario comprimido
    np.save(nombre_archivo_ecg, vector3)

Se definen los parametros de ventana del dataset. Ademas, se fijan los limites para determinar la clasificacion en preictal o no preictal.
Para el ejemplo, 

In [ ]:
# %% IMPORTACIONES Y DEFINICIONES
import pyhrv.tools as tools
import pyhrv.time_domain as td 
import pyhrv.frequency_domain as fd
import pyhrv.nonlinear as nl 
import numpy as np

LI_periodo_preictal = 15 #limite inferior en minutos. "minutos antes de comenzar la crisis"
LS_periodo_preictal = 0 #limite superior en minutos. "minutos de la crisis que incluyo"
ventana_len = 180
ventana_solap = 60
nombre_archivo_excell = 'vent180_solap60_15minpreictal.xlsx'

Base utilizada: https://physionet.org/content/siena-scalp-eeg/1.0.0/

esta base de datos contiene las se;ales en archivos .EDF y un archivo txt donde se detalla el momento de la crisis y el canal donde se registro el ECG.
Se recorren los .txt extrayendo la informacion para los pacientes. Para el primer paciente:

Archivo txt: "Seizures-list-PN00"


PN00

Data Sampling Rate: 512 Hz

Channels in EDF files:
Channel 1: Fp1      
Channel 2: F3      
Channel 3: C3      
Channel 4: P3      
Channel 5: 1      
Channel 6: F7      
Channel 7: T3      
Channel 8: T5      
Channel 9: Fc1      
Channel 10: Fc5      
Channel 11: Cp1      
Channel 12: Cp5      
Channel 13: F9      
Channel 14: Fz      
Channel 15: Cz      
Channel 16: Pz      
Channel 17: Fp2      
Channel 18: F4      
Channel 19: C4      
Channel 20: P4      
Channel 21: O2      
Channel 22: F8      
Channel 23: T4      
Channel 24: T6      
Channel 25: Fc2      
Channel 26: Fc6      
Channel 27: Cp2      
Channel 28: Cp6      
Channel 29: F10      
Channel 33: EKG 1    
Channel 34: EKG 2


Seizure n 1
File name: PN00-1.edf
Registration start time: 19.39.33
Registration end time:  20.22.58
Seizure start time: 19.58.36
Seizure end time: 19.59.46

Seizure n 2 
File name: PN00-2.edf
Registration start time:  02.18.17
Registration end time:  02.56.19
Seizure start time: 02.38.37
Seizure end time: 02.39.31

Seizure n 3
File name: PN00-3.edf
Registration start time: 18.15.44
Registration end time: 18.57.13
Seizure start time: 18.28.29
Seizure end time: 19.29.29

Seizure n 4
File name: PN00-4.edf
Registration start time: 20.51.43
Registration end time: 21.26.25
Seizure start time: 21.08.29
Seizure end time: 21.09.43

Seizure n 5 
File name: PN00-5.edf
Registration start time: 22.22.04
Registration end time:  22.57.27
Seizure start time: 22.37.08
Seizure end time: 22.38.15

NOTA: El dato de en que canal se guarda la señal de ECG no siempre es correcto.

Se extraen los datos e ingresan al codigo:


In [ ]:
# %% PACIENTE 0 SEIZURE 1
# ingreso datos manualmente
frecuencia_muestreo = 512
canal = 33
nombre = 'D:\database Tesis\siena-scalp-eeg-database-1.0.0\siena-scalp-eeg-database-1.0.0\PN00\PN00-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "19.39.33"
hora_inicio_crisis = "19.58.36"
hora_fin_crisis = "19.59.46"
hora_fin_registro = "20.22.58"
nombre_paciente = "P0_SZ1"

Se calcula el numero de muestra en el que inicio y termino la crisis. 


In [ ]:
segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo


Se calcula el numero de muestra en el que la señal comienza a ser "preictal".

In [ ]:
muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


esto creo que no se usa

In [ ]:
muestra_inicial_interictal = 0
muestra_final_interictal = 0

Defino los vectores que van a guardar las ventanas y la clasificacion de cada una.
La funcion "ventanear_1" es apta para trabajar con archivos EDF en los que solo hay una crisis presente. 
Si en el archivo EDF hay presentes dos crisis, trabajar con "ventanear_1". Si hay tres crisis, utilizar "ventanear_3".

In [ ]:
clasificacion = []
ventanas_rpeaks = []
ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

En este momento, ventanas_rpeaks es una matriz de (ventana_len)x(cantidad ventanas).
clasificacion es un vector que contiene la clasificacion de cada ventana.
Ejemplo: 
clasificacion[0] devuelve no_preictal.
ventanas_rpeaks[0] devuelve 180 valores, donde cada valor es el numero de muestra en la que se detecto el latido del corazon.


Ahora debo transformar el contenido de ventanas_rpeaks, ya que la biblioteca utilizada necesita el tiempo en milisegundos entre latido y latido. El numero de meustra en que se detecto no me sirve.


Se procede de la siguiente manera:
con np.diff realizo la resta entre valores consecutivos.
Luego, a esa distancia de muestras entre latidos, la transformo a ms.

In [ ]:
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):
    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista



Ahora ventanas_intervalos_nn es un vector de (ventana_len-1)x(cantidad_ventanas).
ventanas_intervalos_nn[0] devuelve un vector de ventana_len-1 posiciones, cada posicion es un valor en milisegundos que representa el tiempo entre un latido y el siguiente:
ventanas_intervalos_nn[0][0] devuelve el tiempo entre el primer latido del corazon y el segundo latido del corazon.
ventanas_intervalos_nn[0][1] devuelve el tiempo entre el segundo latido del corazon y el tercer latido del corazon.

Se calculan los parametros de la señal:
nni_diff_mean
hr_mean
hr_max
hr_std
pnn50
VLF_power
LF_power
HF_power
sd1
sd2
sd_ratio
elipse_area
SampEn


Se mantiene una columna de clasificacion:
clasificacion

Las siguientes columnas son calculadas desde el mismo programa Excel:
CSI ()
LF_HF_ratio ()

In [ ]:
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

Resultado: Asi hemos calculado la hoja "P0_SZ1" del archivo "dataset6".

| nni_diff_mean | hr_mean    | hr_max     | hr_std     | pnn50      | VLF_power   | LF_power   | HF_power   | sd1        | sd2        | sd_ratio   | elipse_area | SampEn     | clasificacion |
|---------------|------------|------------|------------|------------|-------------|------------|------------|------------|------------|------------|-------------|------------|---------------|
| 89,39387289   | 64,32048102| 125,3877551| 9,365875128| 48,31460674| 931,0771943 | 1002,027032| 2532,889384| 100,5254901| 104,1216612| 1,035773724| 32882,67651 | 0,8998777022| no_preictal   |
| 92,54301264   | 63,2440668 | 95,10835913| 7,131630289| 50         | 449,7501804 | 690,7541911| 3538,526249| 101,7153886| 75,59808693| 0,74323156 | 24157,2415  | 1,033671734 | no_preictal   |
| 82,03125      | 62,48636032| 89,30232558| 6,398223126| 46,62921348| 448,0657196 | 587,6368695| 3578,251317| 91,67223957| 74,41121547| 0,811709366| 21430,1945  | 0,788349897 | no_preictal   |
| 70,43319874   | 62,1354692 | 89,30232558| 6,315636042| 40,4494382 | 189,1909916 | 519,3712639| 2452,037491| 86,93882008| 73,02693086| 0,839980699| 19945,5797  | 0,573065587 | preictal      |
| 75,98534059   | 64,30737183| 91,42857143| 7,353413477| 37,64044944| 2297,534263 | 436,8386184| 2577,924201| 91,54339588| 98,24297854| 1,073184773| 28253,90057 | 0,868219126 | preictal      |
| 86,18987008   | 63,64659213| 91,42857143| 7,256523955| 49,43820225| 459,7989252 | 457,2665107| 3642,834335| 94,35141851| 95,37360358| 1,010833807| 28270,04493 | 0,886542256 | preictal      |
| 78,20180829   | 63,47853528| 91,42857143| 6,392689297| 46,06741573| 557,2354445 | 563,8810434| 2907,443613| 84,47722579| 86,61546292| 1,025311403| 22987,14032 | 0,891160067 | preictal      |
| 73,69206461   | 63,62145643| 85,33333333| 6,427205363| 39,3258427 | 1176,63297  | 582,6348313| 3461,591949| 84,58915894| 87,45151124| 1,033838288| 23239,7739  | 0,903262601 | preictal      |
| 92,36745084   | 63,28807576| 89,30232558| 7,014209383| 49,43820225| 528,920004  | 951,5205255| 4277,14913 | 99,41773803| 79,65496581| 0,801214827| 24878,63829 | 1,110629601 | preictal      |
| 67,09752458   | 68,56703428| 106,6666667| 13,00845375| 37,07865169| 1621,230703 | 990,0096532| 2726,092812| 78,34281748| 175,4775725| 2,239868033| 43188,75421 | 0,645195887 | preictal      |
| 18,69733146   | 88,10048537| 116,3636364| 18,10503998| 7,865168539| 28172,13818 | 483,1984716| 158,1983717| 31,32733289| 221,0498385| 7,056133353| 21755,22086 | 0,196072728 | no_preictal   |
| 38,81012992   | 81,06858265| 188,4662577| 11,61223457| 20,78651685| 1806,142122 | 727,4213406| 998,3079459| 56,78113324| 89,39201795| 1,574326063| 15946,03342 | 0,63843268  | no_preictal   |
| 104,3714888   | 70,93852419| 96,90851735| 9,363110676| 53,37078652| 1034,331804 | 677,8279506| 5806,629416| 108,7607662| 116,2351393| 1,068723064| 39715,45765 | 0,897161562 | no_preictal   |
| 105,3151334   | 64,02857753| 94,52307692| 8,341418048| 56,17977528| 681,5471687 | 660,2002712| 5591,99587 | 110,9651939| 101,7254387| 0,916732853| 35462,24456 | 0,640230555 | no_preictal   |
| 123,9905197   | 63,70407991| 96,60377358| 8,732896428| 65,73033708| 595,9609425 | 1427,335053 | 6030,026959| 122,8880493| 91,95362618| 0,74827151 | 35500,00246 | 0,797554035 | no_preictal   |
| 103,3839537   | 63,77008449| 92,53012048| 7,084476586| 63,48314607| 729,7627264 | 879,5688154| 3928,891724| 101,8156097| 81,36156772| 0,799107012| 26024,5691  | 1,152100974 | no_preictal   |
| 90,88614817   | 64,65206891| 91,70149254| 6,973383755| 50         | 1040,670019 | 1302,275321| 4060,900982| 96,1181844 | 82,22810289| 0,855489556| 24829,94183 | 1,280933845 | no_preictal   |
| 98,65475772   | 65,23451854| 92,80966767| 7,786911139| 47,19101124| 726,371869  | 564,4795798| 3586,860532| 106,7792298| 88,43512684| 0,828205326| 29666,16853 | 1,107233032 | no_preictal   |
| 108,6837254   | 65,03716427| 100,3921569| 8,558994728| 55,05617978| 853,0609871 | 764,7050449| 6027,706047| 114,2908317| 98,59825382| 0,862696091| 35402,21943 | 0,943974161 | no_preictal   |
| 93,5744382    | 63,38297999| 88,78612717| 7,213865489| 49,43820225| 1408,142331 | 856,9358618| 2513,481622| 101,37944  | 82,86963621| 0,817420537| 26393,39108 | 0,95219744  | no_preictal   |
| 87,16643258   | 64,88183521| 94,81481481| 7,298909952| 42,69662921| 1371,747805 | 582,0781618| 2623,436503| 96,97784507| 81,15870461| 0,836878821| 24726,20746 | 1,460664904 | no_preictal   |
| 71,1135007    | 63,36729505| 94,23312883| 6,351594692| 41,01123596| 1271,7148   | 532,3223413| 1488,322122| 83,38211986| 80,20148653| 0,961854732| 21008,99235 | 0,779665756 | no_preictal   |
| 78,04819171   | 62,524391  | 91,42857143| 6,766250723| 43,25842697| 1312,214071 | 746,1771052| 2664,45663 | 92,291121  | 79,11310923| 0,857212572| 22938,14253 | 0,60709539  | no_preictal   |



El proceso se repite para los siguientes pacientes, llenando cada archivo .EDF una nueva hoja al libro de Excel.
(Esta es la ultima celda que contiene texto)


In [ ]:

# %% PACIENTE 0 SEIZURE 2
frecuencia_muestreo = 512
canal = 33
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN00/PN00-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "02.18.17"
hora_inicio_crisis = "02.38.37"
hora_fin_crisis = "02.39.31"
hora_fin_registro = "02.56.19"
nombre_paciente = "P0_SZ2"

segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []

ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):
    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista


calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 0 SEIZURE 3 - descartado por no tener sentido
# frecuencia_muestreo = 512
# canal = 33
# nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN00/PN00-3.edf'
# rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
# hora_inicio_registro = "18.15.44"
# hora_inicio_crisis = "18.28.29"
# hora_fin_crisis = "19.29.29"
# hora_fin_registro = "18.57.13"
# nombre_paciente = "P0_SZ3"


# segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
# segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
# segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
# muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
# muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

# muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
# muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 

# muestra_inicial_interictal = 0
# muestra_final_interictal = 0

# clasificacion = []
# ventanas_rpeaks = []

# ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

# ventanas_ms = []
# time_domain_features = []
# freq_features = []
# nonlinear_features = []
# nn_intervals = []
# ventanas_intervalos_nn = []

# for i in range(len(ventanas_rpeaks)):
#     diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
#     ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista


# calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)


# %% PACIENTE 0 SEIZURE 4
frecuencia_muestreo = 512
canal = 33
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN00/PN00-4.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "20.51.43"
hora_inicio_crisis = "21.08.29"
hora_fin_crisis = "21.09.43"
hora_fin_registro = "21.26.25"
nombre_paciente = "P0_SZ4"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 

muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []

ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []




for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista

calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 0 SEIZURE 5

frecuencia_muestreo = 512
canal = 33
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN00/PN00-5.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "22.22.04"
hora_inicio_crisis = "22.37.08"
hora_fin_crisis = "22.38.15"
hora_fin_registro = "22.57.27"
nombre_paciente = "P0_SZ5"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 

muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []

ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista

calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 1 - dudoso el ECG, lo suspendo 

# frecuencia_muestreo = 512
# canal = 32
# nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN01/PN01-1.edf'
# rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
# hora_inicio_registro = "19.00.44"
# hora_inicio_crisis1 = "21.51.02"
# hora_fin_crisis1 = "21.51.56"
# hora_inicio_crisis2 = "07.53.17"
# hora_fin_crisis2 = "07.53.17"
# hora_fin_registro = "08.29.41"
# nombre_paciente = "P1"


# segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
# segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)

# segundos_inicio_crisis2 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis2, False)
# segundos_fin_crisis2 = calcular_segundos(hora_inicio_registro, hora_fin_crisis2, False)

# segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, False)

# muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
# muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

# muestra_inicial_crisis2 = segundos_inicio_crisis2 * frecuencia_muestreo
# muestra_final_crisis2 = segundos_fin_crisis2 * frecuencia_muestreo

# muestra_inicial_preictal1 = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
# muestra_final_preictal1 = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 

# muestra_inicial_preictal2 = (segundos_inicio_crisis2 - LI_periodo_preictal*60) * frecuencia_muestreo
# muestra_final_preictal2 = (segundos_fin_crisis2 + LS_periodo_preictal*60) * frecuencia_muestreo
# clasificacion = []
# ventanas_rpeaks = []
# ventanas_rpeaks, clasificacion = ventanear_2(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal1, muestra_final_preictal1, muestra_inicial_preictal2, muestra_final_preictal2)

# ventanas_ms = []
# time_domain_features = []
# freq_features = []
# nonlinear_features = []
# nn_intervals = []
# ventanas_intervalos_nn = []

# for i in range(len(ventanas_rpeaks)):

#     diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
#     ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista

# calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
# %% PACIENTE 3 SEIZURE 1



frecuencia_muestreo = 512
canal = 28
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN03/PN03-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "22.44.37"
hora_inicio_crisis = "09.29.10"
hora_fin_crisis = "09.31.01"
hora_fin_registro = "11.41.34"
nombre_paciente = "P3_SZ1"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, False)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, False)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, False)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 

muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []

ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista

calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 3 SEIZURE 2

frecuencia_muestreo = 512
canal = 28
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN03/PN03-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "21.31.04"
hora_inicio_crisis = "07.13.05"
hora_fin_crisis = "07.15.18"
hora_fin_registro = "08.47.04"
nombre_paciente = "P3_SZ2"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, False)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, False)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, False)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 

muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista

calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)


vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 5 SEIZURE 2
frecuencia_muestreo = 512
canal = 31
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN05/PN05-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "06.46.02"
hora_inicio_crisis = "08.45.25"
hora_fin_crisis = "08.46.00"
hora_fin_registro = "09.19.47"
nombre_paciente = "P5_SZ2"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista

calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 5 SEIZURE 3

frecuencia_muestreo = 512
canal = 31
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN05/PN05-3.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "06.01.23"
hora_inicio_crisis = "07.55.19"
hora_fin_crisis = "07.55.49"
hora_fin_registro = "08.06.57"
nombre_paciente = "P5_SZ3"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista

calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 5 SEIZURE 4
ventana_len = 180
ventana_solap = 60


frecuencia_muestreo = 512
canal = 31
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN05/PN05-4.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "06.38.35"
hora_inicio_crisis = "07.38.43"
hora_fin_crisis = "07.39.22"
hora_fin_registro = "08.00.23"
nombre_paciente = "P5_SZ4"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista

calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 6 SEIZURE 1

frecuencia_muestreo = 512
canal = 31
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN06/PN06-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "04.21.22"
hora_inicio_crisis = "05.54.25"
hora_fin_crisis = "05.55.29"
hora_fin_registro = "07.01.37"
nombre_paciente = "P6_SZ1"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 6 SEIZURE 2



frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN06/PN06-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "21.11.29"
hora_inicio_crisis = "23.39.09"
hora_fin_crisis = "23.40.18"
hora_fin_registro = "00.41.49"
nombre_paciente = "P6_SZ2"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, False)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 6 SEIZURE 3



frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN06/PN06-3.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "06.25.51"
hora_inicio_crisis = "08.10.26"
hora_fin_crisis = "08.11.08"
hora_fin_registro = "08.41.51"
nombre_paciente = "P6_SZ3"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 6 SEIZURE 4

frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN06/PN06-4.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "11.16.09"
hora_inicio_crisis = "12.55.08"
hora_fin_crisis = "12.56.11"
hora_fin_registro = "13.12.54"
nombre_paciente = "P6_SZ4"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 6 SEIZURE 5

frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN06/PN06-5.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "13.24.41"
hora_inicio_crisis = "14.44.24"
hora_fin_crisis = "14.45.08"
hora_fin_registro = "15.04.43"
nombre_paciente = "P6_SZ5"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 7



frecuencia_muestreo = 512
canal = 28
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN07/PN07-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "23.18.10"
hora_inicio_crisis = "05.25.49"
hora_fin_crisis = "05.26.51"
hora_fin_registro = "08.01.58"
nombre_paciente = "P7"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, False)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, False)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, False)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 9 SEIZURE 1

frecuencia_muestreo = 512
canal = 31
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN09/PN09-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "14.08.54"
hora_inicio_crisis = "16.09.43"
hora_fin_crisis = "16.11.03"
hora_fin_registro = "16.26.07"
nombre_paciente = "P9_SZ1"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo


muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 9 SEIZURE 2

frecuencia_muestreo = 512
canal = 31
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN09/PN09-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "15.02.09"
hora_inicio_crisis = "17.00.56"
hora_fin_crisis = "17.01.55"
hora_fin_registro = "17.21.02"
nombre_paciente = "P9_SZ2"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo
muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 9 SEIZURE 3

frecuencia_muestreo = 512
canal = 28
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN09/PN09-3.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "14.20.23"
hora_inicio_crisis = "16.20.44"
hora_fin_crisis = "16.21.48"
hora_fin_registro = "16.34.28"
nombre_paciente = "P9_SZ3"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo
muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 10 SEIZURE 1

frecuencia_muestreo = 512
canal = 28
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN10/PN10-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "05.40.05"
hora_inicio_crisis = "07.45.50"
hora_fin_crisis = "07.46.59"
hora_fin_registro = "08.26.07"
nombre_paciente = "P10_SZ1"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 10 SEIZURE 2

frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN10/PN10-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "09.30.15"
hora_inicio_crisis = "11.40.13"
hora_fin_crisis = "11.41.04"
hora_fin_registro = "11.52.29"
nombre_paciente = "P10_SZ2"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 10 SEIZURE 3

frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN10/PN10-3.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "13.33.18"
hora_inicio_crisis = "15.43.53"
hora_fin_crisis = "15.45.02"
hora_fin_registro = "15.58.31"
nombre_paciente = "P10_SZ3"


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 10 SEIZURE 4, 5, 6
ventana_len = 180
ventana_solap = 60


frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN10/PN10-4.5.6.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "12.11.21"

hora_inicio_crisis1 = "12.49.50"
hora_fin_crisis1 = "12.49.55"

hora_inicio_crisis2 = "14.00.25"
hora_fin_crisis2 = "14.00.44"

hora_inicio_crisis3 = "15.18.26"
hora_fin_crisis3 = "15.19.23"

hora_fin_registro = "16.48.49"
nombre_paciente = "P10_SZ456"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal1 = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal1 = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 



segundos_inicio_crisis2 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis2, True)
segundos_fin_crisis2 = calcular_segundos(hora_inicio_registro, hora_fin_crisis2, True)
muestra_inicial_crisis2 = segundos_inicio_crisis2 * frecuencia_muestreo
muestra_final_crisis2 = segundos_fin_crisis2 * frecuencia_muestreo
muestra_inicial_preictal2 = (segundos_inicio_crisis2 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal2 = (segundos_fin_crisis2+LS_periodo_preictal*60) * frecuencia_muestreo 



segundos_inicio_crisis3 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis3, True)
segundos_fin_crisis3 = calcular_segundos(hora_inicio_registro, hora_fin_crisis3, True)
muestra_inicial_crisis3 = segundos_inicio_crisis3 * frecuencia_muestreo
muestra_final_crisis3 = segundos_fin_crisis3 * frecuencia_muestreo
muestra_inicial_preictal3 = (segundos_inicio_crisis3 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal3 = (segundos_fin_crisis3+LS_periodo_preictal*60) * frecuencia_muestreo

ventanas_rpeaks, clasificacion = ventanear_3(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal1, muestra_final_preictal1, muestra_inicial_preictal2, muestra_final_preictal2,muestra_inicial_preictal3, muestra_final_preictal3)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1,muestra_inicial_crisis2, muestra_final_crisis2,muestra_inicial_crisis3, muestra_final_crisis3]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 10 SEIZURE 7, 8, 9

frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN10/PN10-7.8.9.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "16.49.25"

hora_inicio_crisis1 = "17.35.13"
hora_fin_crisis1 = "17.36.01"

hora_inicio_crisis2 = "18.20.24"
hora_fin_crisis2 = "18.20.42"

hora_inicio_crisis3 = "20.24.48"
hora_fin_crisis3 = "20.25.03"

hora_fin_registro = "20.58.40"
nombre_paciente = "P10_SZ789"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal1 = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal1 = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 



segundos_inicio_crisis2 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis2, True)
segundos_fin_crisis2 = calcular_segundos(hora_inicio_registro, hora_fin_crisis2, True)
muestra_inicial_crisis2 = segundos_inicio_crisis2 * frecuencia_muestreo
muestra_final_crisis2 = segundos_fin_crisis2 * frecuencia_muestreo
muestra_inicial_preictal2 = (segundos_inicio_crisis2 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal2 = (segundos_fin_crisis2+LS_periodo_preictal*60) * frecuencia_muestreo 



segundos_inicio_crisis3 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis3, True)
segundos_fin_crisis3 = calcular_segundos(hora_inicio_registro, hora_fin_crisis3, True)
muestra_inicial_crisis3 = segundos_inicio_crisis3 * frecuencia_muestreo
muestra_final_crisis3 = segundos_fin_crisis3 * frecuencia_muestreo
muestra_inicial_preictal3 = (segundos_inicio_crisis3 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal3 = (segundos_fin_crisis3+LS_periodo_preictal*60) * frecuencia_muestreo

ventanas_rpeaks, clasificacion = ventanear_3(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal1, muestra_final_preictal1, muestra_inicial_preictal2, muestra_final_preictal2,muestra_inicial_preictal3, muestra_final_preictal3)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1,muestra_inicial_crisis2, muestra_final_crisis2,muestra_inicial_crisis3, muestra_final_crisis3]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 10 SEIZURE 10

frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN10/PN10-10.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "08.45.22"

hora_inicio_crisis1 = "10.58.19"
hora_fin_crisis1 = "10.58.33"


hora_fin_registro = "11.08.07"
nombre_paciente = "P10_SZ10"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)


segundos_inicio_crisis = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)
muestra_inicial_crisis = segundos_inicio_crisis * frecuencia_muestreo
muestra_final_crisis = segundos_fin_crisis * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis, muestra_final_crisis]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 11 



frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN11/PN11-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "11.31.25"

hora_inicio_crisis1 = "13.37.19"
hora_fin_crisis1 = "13.38.14"


hora_fin_registro = "13.56.02"
nombre_paciente = "P11"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo
muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal1, muestra_final_preictal1)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 12 SEIZURE 1, 2



frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN12/PN12-1.2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "15.51.31"

hora_inicio_crisis1 = "16.13.23"
hora_fin_crisis1 = "16.14.26"

hora_inicio_crisis2 = "18.31.01"
hora_fin_crisis2 = "18.32.09"

hora_fin_registro = "18.34.25"
nombre_paciente = "P12_SZ1"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo
muestra_inicial_preictal1 = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal1 = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo


segundos_inicio_crisis2 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis2, True)
segundos_fin_crisis2 = calcular_segundos(hora_inicio_registro, hora_fin_crisis2, True)
muestra_inicial_crisis2 = segundos_inicio_crisis2 * frecuencia_muestreo
muestra_final_crisis2 = segundos_fin_crisis2 * frecuencia_muestreo
muestra_inicial_preictal2 = (segundos_inicio_crisis2 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal2 = (segundos_fin_crisis2+LS_periodo_preictal*60) * frecuencia_muestreo

ventanas_rpeaks, clasificacion = ventanear_2(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal1, muestra_final_preictal1, muestra_inicial_preictal2, muestra_final_preictal2)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1, muestra_inicial_crisis2, muestra_inicial_crisis3]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 12 SEIZURE 3


frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN12/PN12-3.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "08.42.35"

hora_inicio_crisis1 = "08.55.27"
hora_fin_crisis1 = "08.57.03"

hora_fin_registro = "09.15.44"
nombre_paciente = "P12_SZ3"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)

vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)
# %% PACIENTE 12 SEIZURE 4


frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN12/PN12-4.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "15.59.19"

hora_inicio_crisis1 = "18.42.51"
hora_fin_crisis1 = "18.43.54"

hora_fin_registro = "18.49.13"
nombre_paciente = "P12_SZ4"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo
muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 13 SEIZURE 1


frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN13/PN13-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "08.24.28"

hora_inicio_crisis1 = "10.22.10"
hora_fin_crisis1 = "10.22.58"

hora_fin_registro = "11.00.23"
nombre_paciente = "P13_SZ1"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 13 SEIZURE 2


frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN13/PN13-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "06.55.02"

hora_inicio_crisis1 = "08.55.51"
hora_fin_crisis1 = "08.56.56"

hora_fin_registro = "09.30.14"
nombre_paciente = "P13_SZ2"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 13 SEIZURE 3


frecuencia_muestreo = 512
canal = 32
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN13/PN13-3.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "12.00.01"

hora_inicio_crisis1 = "14.05.54"
hora_fin_crisis1 = "14.08.25"

hora_fin_registro = "15.28.48"
nombre_paciente = "P13_SZ3"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 14 SEIZURE 1


frecuencia_muestreo = 512
canal = 29
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN14/PN14-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "11.44.58"

hora_inicio_crisis1 = "13.46.00"
hora_fin_crisis1 = "13.46.27"

hora_fin_registro = "13.56.06"
nombre_paciente = "P14_SZ1"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 14 SEIZURE 2


frecuencia_muestreo = 512
canal = 29
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN14/PN14-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "15.50.13"

hora_inicio_crisis1 = "17.54.52"
hora_fin_crisis1 = "17.55.04"

hora_fin_registro = "18.22.58"
nombre_paciente = "P14_SZ2"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 14 SEIZURE 3


frecuencia_muestreo = 512
canal = 29
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN14/PN14-3.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "16.17.45"

hora_inicio_crisis1 = "21.10.05"
hora_fin_crisis1 = "21.10.46"

hora_fin_registro = "06.57.40"
nombre_paciente = "P14_SZ3"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, False)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 14 SEIZURE 4


frecuencia_muestreo = 512
canal = 29
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN14/PN14-4.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "14.18.30"

hora_inicio_crisis1 = "15.49.33"
hora_fin_crisis1 = "15.50.56"

hora_fin_registro = "18.22.32"
nombre_paciente = "P14_SZ4"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 16 SEIZURE 1


frecuencia_muestreo = 512
canal = 28
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN16/PN16-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "20.45.21"

hora_inicio_crisis1 = "22.45.05"
hora_fin_crisis1 = "22.47.08"

hora_fin_registro = "23.03.11"
nombre_paciente = "P16_SZ1"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 16 SEIZURE 2


frecuencia_muestreo = 512
canal = 28
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN16/PN16-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "00.53.55"

hora_inicio_crisis1 = "03.16.49"
hora_fin_crisis1 = "03.18.36"

hora_fin_registro = "03.28.52"
nombre_paciente = "P16_SZ2"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 17 SEIZURE 1


frecuencia_muestreo = 512
canal = 29
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN17/PN17-1.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "20.14.28"

hora_inicio_crisis1 = "22.34.48"
hora_fin_crisis1 = "22.35.58"

hora_fin_registro = "22.49.05"
nombre_paciente = "P17_SZ1"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

# %% PACIENTE 17 SEIZURE 2


frecuencia_muestreo = 512
canal = 29
nombre = '../siena-scalp-eeg-database-1.0.0/siena-scalp-eeg-database-1.0.0/PN17/PN17-2.edf'
rpeaks, eje_temp, ecg = abrir_y_corroborar(nombre, canal, frecuencia_muestreo)
hora_inicio_registro = "13.52.18"

hora_inicio_crisis1 = "16.01.09"
hora_fin_crisis1 = "16.02.32"

hora_fin_registro = "16.25.10"
nombre_paciente = "P17_SZ2"

segundos_fin_registro = calcular_segundos(hora_inicio_registro, hora_fin_registro, True)

segundos_inicio_crisis1 = calcular_segundos(hora_inicio_registro, hora_inicio_crisis1, True)
segundos_fin_crisis1 = calcular_segundos(hora_inicio_registro, hora_fin_crisis1, True)
muestra_inicial_crisis1 = segundos_inicio_crisis1 * frecuencia_muestreo
muestra_final_crisis1 = segundos_fin_crisis1 * frecuencia_muestreo

muestra_inicial_preictal = (segundos_inicio_crisis1 - LI_periodo_preictal*60) * frecuencia_muestreo
muestra_final_preictal = (segundos_fin_crisis1+LS_periodo_preictal*60) * frecuencia_muestreo 


muestra_inicial_interictal = 0
muestra_final_interictal = 0
clasificacion = []
ventanas_rpeaks = []


ventanas_rpeaks, clasificacion = ventanear_1(rpeaks, ecg, ventana_len, ventana_solap, muestra_inicial_preictal, muestra_final_preictal)

ventanas_ms = []
time_domain_features = []
freq_features = []
nonlinear_features = []
nn_intervals = []
ventanas_intervalos_nn = []

for i in range(len(ventanas_rpeaks)):

    diferencias = np.diff(ventanas_rpeaks[i])  # Esto calcula nn_intervals[i][j] - nn_intervals[i][j+1]
    ventanas_intervalos_nn.append(diferencias/frecuencia_muestreo*1000)  # Agregar las diferencias a la lista
calculo_parametros_y_guardar(nombre_archivo_excell, nombre_paciente, ventanas_intervalos_nn, clasificacion)
vector_crisis = [muestra_inicial_crisis1, muestra_final_crisis1]
guardar_vectores_en_txt(vector_crisis, rpeaks, ecg, nombre_paciente)

